In [1]:
import pandas as pd, numpy as np
np.set_printoptions(suppress=True)# 关掉科学计数法
import glob
import os
import csv
# 一次性merge多个pct_chg
from functools import reduce
from datetime import datetime, timedelta
import statsmodels.api as sm
from statsmodels import regression

# import yfinance as yf
import tushare as ts
import time, urllib
ts.set_token('8ef5ec61cdd848715c57c11d58dd71da1271f76b2420d2bac8aef123')
pro = ts.pro_api('8ef5ec61cdd848715c57c11d58dd71da1271f76b2420d2bac8aef123')

import matplotlib
import matplotlib.pyplot as plt
# plt.rcParams['font.sans-serif']=['KaiTi']
# plt.rcParams['axes.unicode_minus'] = False

import seaborn as sns
%matplotlib inline
sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=False, rc=None)


from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go

#setting offilne
plotly.offline.init_notebook_mode(connected=True)

# from research_toolbox_all_in_one import *
# from research_toolbox_all_in_one import calc_macd_signals

In [2]:
# 得到行业龙头的tickers信息

ch_db_path = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/CH_database/"
leaders_tickers_df = pd.read_csv("20200309_110_A_shares_industry_leaders.csv")
leaders_tickers = list(leaders_tickers_df['tickers'])

In [3]:
# 得到行业龙头的全市场数据
industry_leaders_mkt_df = get_mkt_data_df(ch_db_path, leaders_tickers, 'trade_date')
# 利用全市场数据得到MACD信号数据
tgt_stocks_macd_signals = calc_macd_signals(industry_leaders_mkt_df, leaders_tickers, 'ts_code', 'string','close')
# 只提取最近一天的MACD信号数据
tgt_most_recent_stocks_macd_signals = get_last_signals_macd(tgt_stocks_macd_signals, 'trade_date','ts_code')
# This could be different from our current position holdings
tgt_most_recent_stocks_macd_signals['TYPE'] = 'TARGET'
tgt_most_recent_stocks_macd_signals.sort_values("ts_code", inplace=True)
# If there are empty positions then don't use this function:
# signal_df = merge_current_pos_with_target_pos(ch_db_path, cur_positions, tgt_most_recent_stocks_macd_signals, \
#                                               'ts_code','trade_date','string','close')

/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/pandas/compat/_optional.py:106: UserWarning:

Pandas requires version '2.6.2' or newer of 'numexpr' (version '2.6.1' currently installed).

/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/quant_research/research_toolbox_all_in_one.py:228: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/quant_research/research_toolbox_all_in_one.py:229: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users

In [4]:
# 提取当天需要操作的数据
df_actions = tgt_most_recent_stocks_macd_signals[(tgt_most_recent_stocks_macd_signals['SIGNAL_ACTION']=="LONG")|\
                                                (tgt_most_recent_stocks_macd_signals['SIGNAL_ACTION']=="SHORT")]

# 得到price_pct数据：
tgt_returns = get_date_price_code_return_list(ch_db_path, leaders_tickers, 'trade_date', 'close', 'ts_code')
tgt_merged_returns = merge_dfs_by_ticker(tgt_returns, "trade_date")
# 计算仓位比重：
df_wts, stocks, weights = get_df_wts(tgt_merged_returns)
# draw_risk_parity_pie(weights, stocks)
df_wts.columns = ['ts_code','weights']
# df_wts.sort_values("weights", ascending=False)

# 合并仓位数据和信号数据
df_actions_with_weights = df_wts.merge(df_actions, on ='ts_code')
# 仓位太小，创建2倍仓位信息
df_actions_with_weights['weights_enlarged'] = df_actions_with_weights['weights']*2
# 提取下一日要操作的tickers
tickers_to_trade = list(df_actions_with_weights['ts_code'])
# 得到这些tickers的收盘价数据
tickers_to_trade_closes = get_date_price_code_df(ch_db_path, tickers_to_trade, 'trade_date', 'close', 'ts_code')
# 创建今日date信息
last_date = str(datetime.now().date())
# 提取最近一天的tickers的收盘价数据
tickers_to_trade_closes_last_date = tickers_to_trade_closes[tickers_to_trade_closes['trade_date'] == last_date]

# 创建最终的信号-仓位指示信息
df_actions_with_weights = df_actions_with_weights.merge(tickers_to_trade_closes_last_date, on = ['trade_date','ts_code'])
account_values = 70000
df_actions_with_weights['tgt_shares'] = account_values*\
                                        df_actions_with_weights['weights_enlarged']/\
                                        df_actions_with_weights['close']
# 展示信号-仓位结果
df_actions_with_weights

,ts_code,weights,trade_date,SIGNAL_STATUS,SIGNAL_ACTION,TYPE,weights_enlarged,close,tgt_shares
0,600352.SH,0.008408,2020-03-12,空头状态,SHORT,TARGET,0.016816,13.43,87.645972
1,002385.SZ,0.008415,2020-03-12,空头状态,SHORT,TARGET,0.016830,7.40,159.204323
2,601668.SH,0.012223,2020-03-12,空头状态,SHORT,TARGET,0.024447,5.37,318.672876
3,600050.SH,0.010576,2020-03-12,空头状态,SHORT,TARGET,0.021152,5.57,265.826427
4,002024.SZ,0.008430,2020-03-12,空头状态,SHORT,TARGET,0.016860,9.33,126.494394
5,600208.SH,0.008855,2020-03-12,空头状态,SHORT,TARGET,0.017709,3.22,384.978912
6,002221.SZ,0.008851,2020-03-12,空头状态,SHORT,TARGET,0.017702,10.02,123.670087
7,000563.SZ,0.007028,2020-03-12,空头状态,SHORT,TARGET,0.014055,3.99,246.582280
8,002822.SZ,0.011523,2020-03-12,空头状态,SHORT,TARGET,0.023046,9.70,166.308180
9,600968.SH,0.011468,2020-03-12,空头状态,SHORT,TARGET,0.022936,2.51,639.660119


In [5]:
df_actions

,trade_date,ts_code,SIGNAL_STATUS,SIGNAL_ACTION,TYPE
97498,2020-03-12,000563.SZ,空头状态,SHORT,TARGET
194794,2020-03-12,000729.SZ,空头状态,SHORT,TARGET
67507,2020-03-12,002024.SZ,空头状态,SHORT,TARGET
95133,2020-03-12,002221.SZ,空头状态,SHORT,TARGET
214685,2020-03-12,002285.SZ,空头状态,SHORT,TARGET
21591,2020-03-12,002385.SZ,空头状态,SHORT,TARGET
114609,2020-03-12,002822.SZ,空头状态,SHORT,TARGET
33183,2020-03-12,600050.SH,空头状态,SHORT,TARGET
78669,2020-03-12,600208.SH,空头状态,SHORT,TARGET
158100,2020-03-12,600313.SH,空头状态,SHORT,TARGET


In [112]:
last_macd_signals[last_macd_signals["SIGNAL_ACTION"]!="NO CHANGE"]

,trade_date,ts_code,SIGNAL_STATUS,SIGNAL_ACTION
35501,2020-03-13,000652.SZ,空头状态,SHORT
37846,2020-03-13,601288.SH,空头状态,SHORT
57975,2020-03-13,000955.SZ,空头状态,SHORT
72392,2020-03-13,600028.SH,空头状态,SHORT
74578,2020-03-13,601989.SH,空头状态,SHORT
101634,2020-03-13,600010.SH,空头状态,SHORT
121207,2020-03-13,002611.SZ,空头状态,SHORT
123668,2020-03-13,600712.SH,空头状态,SHORT
130686,2020-03-13,000089.SZ,空头状态,SHORT
146301,2020-03-13,600221.SH,空头状态,SHORT
